In [1]:
import sys
import pathlib
import os
from skmap.catalog import s3_setup, DataCatalog
from skmap.loader import TiledDataLoader
from skmap.overlay import SpaceOverlay, SpaceTimeOverlay
from skmap.misc import find_files, GoogleSheet, ttprint
import random
import pandas as pd
import time
import skmap_bindings as sb
import numpy as np
import geopandas as gpd

# Check the overlaid data, do some fix
- drop nan values
- assign block IDs to rows, for spatial validation
- mend the land cover survey data

### Drop nan values

In [2]:
folder_path = '/home/xuemeng/work_xuemeng/soc/SoilHealthDataCube/data'

In [3]:
df_ocd = pd.read_parquet(f'{folder_path}/000_data_ocd.pq')
df_old = pd.read_parquet(f'{folder_path}/001_data_overlayed.1213.pq')

print('ocd data with size, ', df_ocd.shape)
print('once overlaid, size: ', df_old.shape)

ocd data with size,  (47304, 9)
once overlaid, size:  (47286, 598)


In [4]:
meta = ['time', 'lat', 'lon', 'hzn_dep', 'id', 'ref', 'nuts0', 'lc_survey','ocd']

# fix issue for crop layer
# all valid values mean that the pixel is cropland
# if a pixel is not cropland, the pixel value is nan
crop_layer = 'cropland.extent_glad.interpolate_p_30m_s_YYYY0101_YYYY1231_eu_epsg.3035_v20240604'
df_old.loc[df_old[crop_layer]>0,crop_layer ] = 100
df_old.loc[df_old[crop_layer]!=100,crop_layer] = 0

drop_cols = []
df_cols = df_old.columns.values.tolist()
var_cols = [col for col in df_cols if col not in meta]
# check invalid values for each covariate
for icol in df_old.columns:
    if icol in meta:
        continue
    # else:
    #     print(icol, df[icol].min(), df[icol].max())
    elif df_old[icol].isna().sum()/len(df_old)>0.005:
        rto = round(df_old[icol].isna().sum()/len(df_old),2)
        print(df_old[icol].isna().sum(), icol, rto)
        drop_cols.append(icol)
        

# for covariate with more than 0.5% invalid values, drop the covariate
df_old = df_old.drop(columns=drop_cols)
print('\ndata size after excluding invalid covs: ', df_old.shape)
# for covariate with less invalid values, drop the rows without complete valid covariate set
var_cols = [col for col in var_cols if col not in drop_cols]
df_old = df_old.dropna(subset = var_cols, how='any')
print('data size after excluding invalid values: ', df_old.shape)

5560 accum.ndvi_glad.landsat.seasconv.m.yearly_p50_30m_s_YYYY0101_YYYY1231_eu_epsg.3035_v20240513 0.12
11268 lgd_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531 0.24
11268 fgd_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531 0.24
19668 fcf_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531 0.42

data size after excluding invalid covs:  (47286, 594)
data size after excluding invalid values:  (46843, 594)


In [5]:
df_old.to_parquet(f'{folder_path}/003_data_overlaid.organized.pq')

### assign block ID to data

In [6]:
tiles = gpd.read_file(f'{folder_path}/002_eu.tiles_epsg.3035.gpkg')
tiles = tiles.rename(columns={'id':'tile_id'})

# dff = pd.read_parquet(f'{folder_path}/003_data_overlaid.organized.pq')
gdf = gpd.GeoDataFrame(df_old, geometry=gpd.points_from_xy(df_old.lon, df_old.lat))
gdf.crs = 'EPSG:4326'
gdf = gdf.to_crs(tiles.crs)

# join spatial
joined_gdf = gpd.sjoin(gdf, tiles, how="left", op='within')
joined_gdf = joined_gdf.drop(columns=['geometry','index_right'])

print(f'data size: ', joined_gdf.shape)

joined_gdf.to_parquet(f'{folder_path}/003_data_overlaid.organized.pq')

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


data size:  (46843, 595)


### mend the land cover data from original dataset

In [7]:
df = pd.read_parquet(f'{folder_path}/003_data_overlaid.organized.pq')
print('data size: ', df.shape)

glc = 'lc_glc.fcs30d_c_30m_s_YYYY0101_YYYY1231_go_epsg.4326_v20231026'

data size:  (46843, 595)


In [8]:
de = pd.read_parquet('/home/xuemeng/work_xuemeng/ai4sh_data.harmo/data_v2/germany_harmonized_l1.pq')
es1 = pd.read_parquet('/home/xuemeng/work_xuemeng/ai4sh_data.harmo/data_v2/spain.ParcelasCOS_harmonized_l1.pq')
es2 = pd.read_parquet('/home/xuemeng/work_xuemeng/ai4sh_data.harmo/data_v2/spain.ParcelasINES_harmonized_l1.pq')
pt = pd.read_parquet('/home/xuemeng/work_xuemeng/ai4sh_data.harmo/data_v2/portugal_harmonized_l1.pq')

md = pd.concat([de,es1,es2,pt])

md['hzn_dep'] = (md['hzn_top']+md['hzn_btm'])/2

meta = ['lat', 'lon', 'time', 'hzn_dep', 'ref', 'nuts0', 'id']
num_cols = ['lat', 'lon', 'time', 'hzn_top', 'hzn_btm']
for col in num_cols:
    md[col] = pd.to_numeric(md[col], errors='coerce')    
    
str_cols = ['id','ref','nuts0']
for col in str_cols:
    md[col] = md[col].astype(str)
    
md = md.drop_duplicates(subset=meta)

merged = df.merge(md[meta + ['lc_survey']], on=meta, how='left', suffixes=('', '_fill'))
merged['lc_survey'] = merged['lc_survey'].combine_first(merged['lc_survey_fill'])
merged.drop(columns=['lc_survey_fill'], inplace=True)

print(merged.shape, df.shape)

(46843, 595) (46843, 595)


In [9]:
# harmmonize code to match lucas code level 1
# LUCAS
lucas_values = [
    'B13', 'C10', 'B16', 'C23', 'C21', 'B11',
    'E20', 'C31', 'B12', 'B82', 'E10', 'A30', 'B41', 'B22', 'B31',
    'B33', 'C33', 'B18', 'E30', 'C32', 'B21', 'B55', 'B32', 'B15',
    'D10', 'B84', 'B52', 'B17', 'B53', 'C22', 'B37', 'B36', 'B74',
    'B81', 'B54', 'D20', 'B75', 'F40', 'B19', 'B35', 'B14', 'B51',
    'B71', 'H12', 'B23', 'B43', 'B34', 'G21', 'B42', 'B73', 'B72',
    'B76', 'B77', 'A22', 'B45', 'B83', 'H11', 'B44', 'F10', 'F30',
    'F20'
]
lucas_level1 = [code[0]+'00' for code in lucas_values]
lucas_code = dict(zip(lucas_values,lucas_level1))

# spain
# spain_code = {'CL':'B00', 'GL':'E00', 'FL':'C00'} #, 'OL':'others'

# # germany
# de_code = {'A':'B00', 'G':'E00', 'SO':'B00'}

# # glance 
# glance_code = {'Rock':'F00', 'Beach/sand':'F00'}

# # portugal
# pt_code = {'Rainfed arable crop': 'B00',
#            'Mixed crops': 'B00',
#            'Fallow':'B00',
#            'Pine forest': 'C00',
#            'Horticulture':'B00',
#            'Forest': 'C00',
#            'Mediterranean woodland': 'C00',
#            'Pasture': 'E00',
#            'Quercus forest': 'C00',
#            'Vineyard': 'B00',
#            'Irrigated arable crop': 'B00',
#            'Olive grove':'B00',
#            'Eucalypt forest': 'C00',
#            'Fruit trees': 'B00',
#            'Cedars': 'C00',
#            'Sugar beet': 'B00',
#            'Melon': 'B00',
#            'Cotton': 'B00',
#            'Golf course': 'E00'}

code_dict = {**lucas_code, **spain_code, **de_code, **glance_code, **pt_code}

merged['lc_survey'] = merged['lc_survey'].map(code_dict)

In [10]:
# then convert code to names
survey_dict = {
    "A00": "artificial land",
    "B00": "cropland",
    "C00": "woodland",
    "D00": "shrubland",
    "E00": "grassland",
    "F00": "bare land & lichens/moss",
    "G00": "water areas & wetland", # water areas
    "H00": "water areas & wetland", #"wetland",
}
merged['lc_survey'] = merged['lc_survey'].map(survey_dict)

In [11]:
merged = merged.drop(columns=[glc])
v_num = merged['lc_survey'].isna().sum()
print(f'{v_num} rows with invalid land cover survey data')
print(merged.shape)
merged.to_parquet(f'{folder_path}/003_data_overlaid.organized.pq')

154 rows with invalid land cover survey data
(46843, 594)


In [12]:
meged = pd.read_parquet(f'{folder_path}/003_data_overlaid.organized.pq')

# Mend nuts0 code

In [13]:
import geopandas as gpd
nuts = gpd.read_file('/home/xuemeng/work_xuemeng/ai4sh_data.harmo/raw_data/EU/EU_nuts/NUTS_RG_20M_2021_3035.shp')
nuts = nuts.loc[nuts['LEVL_CODE']==0]

df = pd.read_parquet(f'{folder_path}/003_data_overlaid.organized.pq')

# dff = pd.read_parquet(f'{folder_path}/003_data_overlaid.organized.pq')
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
gdf.crs = 'EPSG:4326'
gdf = gdf.to_crs(nuts.crs)

# join spatial
joined_gdf = gpd.sjoin(gdf, nuts, how="left", op='within')
joined_gdf = joined_gdf.drop(columns=['LEVL_CODE', 'CNTR_CODE', 'NAME_LATN', 'NUTS_NAME','MOUNT_TYPE', 'URBN_TYPE', 'COAST_TYPE', 'FID', 'geometry','index_right'])

# fill in
joined_gdf.loc[joined_gdf['nuts0'].isna(),'nuts0'] = joined_gdf.loc[joined_gdf['nuts0'].isna(),'NUTS_ID']

# mannual mend
joined_gdf.loc[joined_gdf['lat']==61.13332565,'nuts0'] = 'NO'  # norway
joined_gdf.loc[(joined_gdf['nuts0'].isna()) & (joined_gdf['lat']>48),'nuts0'] = 'UA' # ukraine
joined_gdf.loc[joined_gdf['nuts0'].isna(), 'nuts0'] = 'TR' # turkey

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [14]:
joined_gdf = joined_gdf.drop(columns=['NUTS_ID'])
print(joined_gdf.shape)
joined_gdf = joined_gdf.to_parquet(f'{folder_path}/003_data_overlaid.organized.pq')

(46843, 594)


# Organize covs

In [15]:
import json
other_cols = ['time', 'lat', 'lon', 'id', 'ref', 'nuts0', 'lc_survey', 'ocd','tile_id']
cov_cols= []
for col in df.columns:
    if col not in other_cols:
        cov_cols.append(col)

with open(f'{folder_path}/007_cov_all.json', "w") as file:
    json.dump(cov_cols, file)        

# Check data validity

In [16]:
df = pd.read_parquet(f'{folder_path}/003_data_overlaid.organized.pq')

In [17]:
m = df.loc[(df['ocd']==0) & (df['lc_survey']!= 'bare land & lichens/moss') & (df['hzn_dep']<30)]
grp = m.groupby(['ref', 'lc_survey']).size().reset_index(name='count')
grp

,ref,lc_survey,count
0,ParcelasINES,cropland,567
1,ParcelasINES,grassland,337
2,ParcelasINES,woodland,315


In [18]:
df = df.loc[~((df['ocd'] == 0) & (df['lc_survey'] != 'bare land & lichens/moss') & (df['hzn_dep']<30))]
print('data size after palusibility check', df.shape)
df.to_parquet(f'{folder_path}/003_data_overlaid.organized.pq')

data size after palusibility check (45616, 594)
